In [12]:
import random
import pygame
import math
from scipy.stats import norm
import time


pygame.init()
screen = pygame.display.set_mode((640, 480))


def simple_movement(x, y, s_x, s_y):
    return int(x+s_x), int(y+s_y)


def sin_movement(x, y, s_x, s_y):
    return int(x+s_x), int(y+math.sin((x+s_x)/screen.get_width()*2*math.pi*s_y)*my_R.get_height())


def Brownian_movement(x, y, s, delta, dt):
    return int(x + s*norm.rvs(delta**2*dt)), int(y + s*norm.rvs(delta**2*dt))


def random_color():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    return (red, green, blue)


def draw_tree(x, y):
    # tree trunk (50 wide and 100 tall)
    pygame.draw.rect(screen, (117, 90, 0), (x, y-100, 50, 100))
    # leaves are a circle
    pygame.draw.circle(screen, (27, 117, 0), (x+25, y-120), 50)


def draw_house(x, y):
    # pink house
    pygame.draw.rect(screen, (255, 171, 244), (x, y-180, 200, 180))
    # brown door
    pygame.draw.rect(screen, (89, 71, 0), (x+80, y-60, 40, 60))
    # yellow door knob
    pygame.draw.circle(screen, (255, 204, 0), (x+112, y-30), 4)
    # triangle roof
    pygame.draw.polygon(screen, (125, 125, 125), ((
        x, y-180), (x+100, y-250), (x+200, y-180)))
    draw_window(x+20, y-90)
    draw_window(x+130, y-90)
    draw_window(x+20, y-20)
    draw_window(x+130, y-20)


def draw_window(x, y):
    # glass
    pygame.draw.rect(screen, random_color(), (x, y-50, 50, 50))
    # frame
    pygame.draw.rect(screen, (0, 0, 0), (x, y-50, 50, 50), 5)
    pygame.draw.rect(screen, (0, 0, 0), (x+23, y-50, 5, 50))
    pygame.draw.rect(screen, (0, 0, 0), (x, y-27, 50, 5))


# this function is able to draw clouds of different sizes
def draw_cloud(x, y, size):
    # put int() around any multiplications by decimals to get rid of this warning:
    # DeprecationWarning: integer argument expected, got float
    pygame.draw.circle(screen, (255, 255, 255), (x, y), int(size*.5))
    pygame.draw.circle(screen, (255, 255, 255),
                       (int(x+size*.5), y), int(size*.6))
    pygame.draw.circle(screen, (255, 255, 255),
                       (x+size, int(y-size*.1)), int(size*.4))


def draw_butterfly(x, y, size):
    width = size
    length = 3*width
    pygame.draw.polygon(screen, random_color(), [(x + width/2, y + length/2), (x - width, y + length/2), (x - 3 * width/2, y - 3 * width/2), (x - width, y - 3*width/2)])
    pygame.draw.polygon(screen, random_color(), [(x + width/2, y + length/2), (x + 2*width, y + length/2), (x + 5*width/2, y - 3 * width/2), (x + 2*width, y - 3*width/2)])
    pygame.draw.polygon(screen, random_color(), [(x + width/2, y + length/2 + width/2), (x - width, y + length/2 + width/2), (x - 3*width/2, y + length+ 2*width), (x - width, y + length + 2*width)])
    pygame.draw.polygon(screen, random_color(), [(x + width/2, y + length/2 + width/2), (x + 2*width, y + length/2 + width/2), (x + 5*width/2, y + length + 2*width), (x +2*width, y + length + 2*width)])
    pygame.draw.ellipse(screen, (0, 0, 0), (x, y, width, length), 0)
    pygame.draw.line(screen, (0, 0, 0), (x + width/2, y ),(x, y - 2*width), 1)
    pygame.draw.line(screen, (0, 0, 0), (x + width/2, y),(x + width, y - 2*width), 1)

    


trees = list()
for i in range(3):
    x = random.randint(0, 590)
    y = random.randint(400, 480)
    trees.append([x, y])



clouds = list()

for i in range(3):
    x = random.randint(0, 640)
    y = random.randint(0, 200)
    size = random.randint(10, 100)
    speed_x = random.randrange(-10, 10, 6)
    speed_y = 0
    clouds.append([x, y, size, speed_x, speed_y])


butterflies = list()

for i in range(3):
    x = random.randint(50, 590)
    y = random.randint(20, 400)
    size = random.randint(10, 30)
    speed = random.randint(10, 20)
    butterflies.append([x, y, size, speed])

delta = 0.25
dt = 0.1

my_time = time.gmtime()
my_pre_sec = my_time[5]

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYUP and event.key == pygame.K_ESCAPE):
            running = False

    pygame.time.delay(17)  # pause for 17 milliseconds


    # the animation logic
    my_time = time.gmtime()
    if my_time[5] != my_pre_sec:
        my_pre_sec = my_time[5]
        for i in range(3):
            (clouds[i][0], clouds[i][1]) = simple_movement(clouds[i][0], clouds[i][1], clouds[i][3], clouds[i][4])
            if clouds[i][0] > screen.get_width() and clouds[i][3] > 0:
                clouds[i][0] = 0
            if clouds[i][0] <= 0 and clouds[i][3] < 0:
                clouds[i][0] = screen.get_width()
    
    for i in range(3):
        (butterflies[i][0], butterflies[i][1]) = Brownian_movement(butterflies[i][0], butterflies[i][1], butterflies[i][3], delta, dt)
        if butterflies[i][0] > screen.get_width():
            butterflies[i][0] = screen.get_width() - butterflies[i][2]
        elif butterflies[i][0] < 0:
            butterflies[i][0] = 0
        if butterflies[i][1] > screen.get_height():
            butterflies[i][1] = screen.get_height() - butterflies[i][2]*3
        elif butterflies[i][1] < 0:
            butterflies[i][1] = 0


    # green ground
    pygame.draw.rect(screen, (0, 160, 3), (0, 400, 640, 80))
    # light blue sky
    pygame.draw.rect(screen, (100, 200, 255), (0, 0, 640, 400))

    draw_house(225, 400)

    draw_tree(60, 400)  # x and y location are the bottom left of tree trunk
    draw_tree(550, 400)
    
    for i in range(3):
        draw_tree(trees[i][0], trees[i][1])
    
    for i in range(3):
        draw_cloud(clouds[i][0], clouds[i][1], clouds[i][2])
        draw_butterfly(butterflies[i][0], butterflies[i][1], butterflies[i][2])

    # update the relavant portions in the display
    pygame.display.update()


pygame.quit()